# Elliptic Curve Integrated Encryption Scheme (ECIES)

In [1]:
import base64
import random
from typing import Tuple
from Crypto.Cipher import AES
import hashlib
import hmac

# Elliptic Curve Form - Twisted Edwards

In [2]:
def is_on_curve(P: Tuple[int, int], p: int):
    x, y = P
    assert ( (a*x*x)+(y*y) ) % p == (1 + d*x*x*y*y) % p

def add_points(P: Tuple[int, int], Q: Tuple[int, int]): 
    """
    Edwards addition formulas
    Ref: sefiks.com/2018/12/26/twisted-edwards-curves
    """
    x1, y1 = P
    x2, y2 = Q
    
    x3 = (((x1*y2 + y1*x2) % p) * pow(1+d*x1*x2*y1*y2, -1, p)) % p
    y3 = (((y1*y2 - a*x1*x2) % p) * pow(1- d*x1*x2*y1*y2, -1, p)) % p
    
    is_on_curve((x3, y3), p)
    
    return x3, y3
    
def apply_double_and_add_method(G: Tuple[int, int], k: int):
    """
    performs k x G fast where G is a point on an elliptic curve
    Ref: sefiks.com/2016/03/27/double-and-add-method/
    """
    target_point = G
    
    k_binary = bin(k)[2:] #0b1111111001 - discard first 2 chars
    
    for i in range(1, len(k_binary)):
        current_bit = k_binary[i: i+1]
        
        # doubling - always
        target_point = add_points(target_point, target_point)
        
        # add base point if it is equal to 1
        if current_bit == "1":
            target_point = add_points(target_point, G)
    
    is_on_curve(target_point, p)
    
    return target_point

# Curve

In [3]:
# curve25519
p = pow(2, 255) - 19

# curve arguments for curve25519
a = -1
# d = -121665/121666
d = -121665 * pow(121666, -1, p)

# base point for curve25519
G = (15112221349535400772501151409588531511454012693041857206046113283949847762202
, 46316835694926478169428394003475163141307993866256225615783033603165251855960)

In [4]:
# validate base point is on curve25519
is_on_curve(G, p)

# Alice generates her private and public key pair

In [5]:
# Alice's private key
ka = random.getrandbits(256)

# Alice's public key
Qa = apply_double_and_add_method(G = G, k = ka)

In [6]:
ka

112870149083640627922774909436250781024307780202325514769567925678767588972712

In [7]:
Qa

(9549143089063042697223239628661024736940666206906515834708689969270296464632,
 52204807015919058933768597571088736428556521267634701921407995846661422840267)

# Bob will encrypt a message and send it to Alice

In [8]:
# Bob is going to send this message to Alice encrypted
msg = "attack tomorrow!"

# Bob generates a random integer and calculates 2 points

In [9]:
# Bob's random key
rb = random.getrandbits(256)

# Point U will be sent to Alice
U = apply_double_and_add_method(G = G, k = rb)

# Point T will keep secret and use for encryption purposes
T = apply_double_and_add_method(G = Qa, k = rb)

In [10]:
rb

81751249801823981867964623507503649987210599900252526846840002074092366439896

In [11]:
U

(50950721826062822139675298288118907415968087726432986571287178857048679687064,
 26912672306608416006808820050479278980714318177594911685651683324551173867059)

In [12]:
T

(161144920723414281828628334350547930452448257108707563815663855898935350171,
 45450734318154808352043578096559688588205658174186655863223258770672925833422)

# Key Derivation Function

In [13]:
# this function will be public
def derive_keys(T):   
    # He uses sha-256 to hash 192-bit x coordinate of point T
    tx, ty = T
    # get x coordinate of point T
    tx_binary = bin(tx)[2:]
    # get its first 192-bit value
    tx_binary_cropped = tx_binary[0:192]
    # restore 192-bit x coordinate
    tx_restored = str(int(tx_binary_cropped, 2))
    # use sha-156 to hash
    hashed_tx = bin(int(hashlib.sha256(str.encode(tx_restored)).hexdigest(), 16))[2:]

    assert len(hashed_tx) == 256

    # split the hash into 128-bit and 128-bit as k1 and k2

    k1 = int(hashed_tx[0:128], 2).to_bytes(16, byteorder='big')
    k2 = int(hashed_tx[128:], 2).to_bytes(16, byteorder='big')
    
    return k1, k2

In [14]:
k1, k2 = derive_keys(T)

In [15]:
k1

b'\xd1<\x14}\xb3\x8a*d\xce"\xa0|\x10\xbd\x1aR'

In [16]:
k2

b'h\x95\xb9\xf4k\xec\x96\xf6\xe0\xf11\xd7\xb2\x88{\xb1'

# Encryption

In [17]:
# Bob uses k1 to encrypt a message m with aes-128 and obtain c
obj_bob = AES.new(k1)
c = base64.b64encode(obj_bob.encrypt(msg))

In [18]:
c

b'zJjqUeBg957NYLcIZOiFjg=='

In [19]:
# Bob chooses HMAC-SHA256 to calculate a message authentication code (MAC) r with key k2
def find_mac(message, key):
    return hmac.new(
        key, 
        message, 
        hashlib.sha256
    ).hexdigest()

r = find_mac(c, k2)

In [20]:
r

'941bf6eb33580bda3afe66ba75fb0d82b9fa8c0a737d2c77e8cb124b70ec0fe9'

In [21]:
# Bob finally sends Alice (U, c, r)
ciphertext = (U, c, r)

In [22]:
ciphertext

((50950721826062822139675298288118907415968087726432986571287178857048679687064,
  26912672306608416006808820050479278980714318177594911685651683324551173867059),
 b'zJjqUeBg957NYLcIZOiFjg==',
 '941bf6eb33580bda3afe66ba75fb0d82b9fa8c0a737d2c77e8cb124b70ec0fe9')

# Alice gets (U, c, r) from Bob

Alice can find Point T even though Bob did not send this to her. Let's remember the calculations Bob performed.

- U = rb x G 
- T = rb x Qa

Public key of Alice was calculated as Qa = ka x G

- T = rb x ka x G

Here, we can replace rb x G to Point U

- T = ka x U

So, if Alice finds ka x U, then she will have Point T even though this is not shared to her

In [23]:
# Alice receives U and finds ka x U
T_prime = apply_double_and_add_method(G = U, k = ka)

In [24]:
T_prime

(161144920723414281828628334350547930452448257108707563815663855898935350171,
 45450734318154808352043578096559688588205658174186655863223258770672925833422)

In [25]:
assert T == T_prime

In [26]:
# Alice uses same key derivation function to obtain k1, k2
k1_prime, k2_prime = derive_keys(T_prime)

In [29]:
# Alice computes MAC using k2 and compares the result with r
assert r == find_mac(c, k2_prime)

In [30]:
# Alice uses 128-bit AES to decrypt c, and he obtains original message
obj_alice = AES.new(k1_prime)
plaintext = obj_alice.decrypt(base64.b64decode(c))

In [31]:
plaintext

b'attack tomorrow!'